# Road Segmentation Project


In [ ]:
# Imports
import math
import os
import re
import cv2
import torch
import numpy as np
import parameters as params
import utils
import trainer
import matplotlib.pyplot as plt
from glob import glob
from random import sample
from PIL import Image
from torch import nn
from sklearn.model_selection import train_test_split

In [ ]:
# Loading data
images = utils.load_all_from_path(os.path.join(params.ROOT_PATH, 'training', 'images'))[:, :, :, :3]
masks = utils.load_all_from_path(os.path.join(params.ROOT_PATH, 'training', 'groundtruth'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_images, val_images, train_masks, val_masks = train_test_split(
    images, masks, test_size=0.1, random_state=42
)
train_patches, train_labels = utils.image_to_patches(train_images, train_masks)
val_patches, val_labels = utils.image_to_patches(val_images, val_masks)

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    # dataset class that deals with loading the data and making it available by index.

    def __init__(self, is_train, device, use_patches=True, resize_to=(400, 400)):
        self.is_train = is_train
        self.device = device
        self.use_patches = use_patches
        self.resize_to=resize_to
        self.x, self.y, self.n_samples = None, None, None
        self._load_data()

    def _load_data(self):  # not very scalable, but good enough for now
        self.x = train_images if self.is_train else val_images
        self.y = train_masks if self.is_train else val_masks
        if self.use_patches:  # split each image into patches
            self.x, self.y = image_to_patches(self.x, self.y)
        elif self.resize_to != (self.x.shape[1], self.x.shape[2]):  # resize images
            self.x = np.stack([cv2.resize(img, dsize=self.resize_to) for img in self.x], 0)
            self.y = np.stack([cv2.resize(mask, dsize=self.resize_to) for mask in self.y], 0)
        self.x = np.moveaxis(self.x, -1, 1)  # pytorch works with CHW format instead of HWC
        self.n_samples = len(self.x)

    def _preprocess(self, x, y):
        # to keep things simple we will not apply transformations to each sample,
        # but it would be a very good idea to look into preprocessing
        return x, y

    def __getitem__(self, item):
        return self._preprocess(utils.np_to_tensor(self.x[item], self.device), utils.np_to_tensor(self.y[[item]], self.device))

    def __len__(self):
        return self.n_samples

In [ ]:
# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(True, device, use_patches=False, resize_to=(params.RESIZE, params.RESIZE))
val_dataset = ImageDataset(False, device, use_patches=False, resize_to=(params.RESIZE, params.RESIZE))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=params.BATCH_SIZE, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=params.BATCH_SIZE, shuffle=True)

# Baselines

## Baseline 1: UNet 
This is the provided baseline U-Net with F1 score of 86%.

## Baseline 2: ResU-Net --> Road Extraction by Deep Residual U-Net
This is the provided baseline U-Net with F1 score of 89%.

In [ ]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [ ]:
# for image segmentation dice loss could be the best first choice
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'patch_acc': trainer.patch_accuracy_fn}
optimizer = torch.optim.Adam(model.parameters())
trainer.train_smp(train_dataloader, val_dataloader, model, loss_fn, metric_fns, optimizer, 100)

In [ ]:
trainer.train_smp(train_dataloader, val_dataloader, model, loss_fn, metric_fns, optimizer, 20)

In [ ]:
utils.create_submission("test", "images",'resunet_submission.csv', model, device)

In [ ]:
import resunet

In [ ]:
model = resunet.ResUnet(3).to(device)
loss_fn = nn.BCELoss()
metric_fns = {'acc': trainer.accuracy_fn, 'patch_acc': trainer.patch_accuracy_fn}
optimizer = torch.optim.Adam(model.parameters())
trainer.train(train_dataloader, val_dataloader, model, loss_fn, metric_fns, optimizer, 100)

In [ ]:
utils.create_submission("test", "images",'resunet_submission.csv', model, device)

# Upgrade 1 - Using Transfer Learning for the Encoder
In the architecture of the U-Net, the encoder is replaced with pretrained VGG16 model.

### Training


## Upgrade 2 - CGAN --> https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8628717
In the paper, they use a simple Unet architecture. I tried transfer learning in this part (did not give a better score).

## Upgrade 2.3 -- CGAN with DCED Framework --> Road Segmentation of Remotely-Sensed Images Using Deep Convolutional Neural Networks with Landscape Metrics and Conditional Random Fields
In this framework, the writers use 4 additional ideas.
1. Using ELU activation function instead of RELU
2. Using Gaussian Smoothing and Connected Component Labeling
3. False Road Object Removal with LMs
4. Road Object Sharpening with CRFs

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [ ]:
model = model.to(device)

In [ ]:
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': accuracy_fn, 'patch_acc': patch_accuracy_fn}
optimizer = torch.optim.Adam(model.parameters())
train(train_dataloader, val_dataloader, model, loss_fn, metric_fns, optimizer, 10)